In [30]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
import pickle

In [31]:
import mlflow

#mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_tracking_uri("http://127.0.0.1:5003") 
mlflow.set_experiment('nyc_greentaxi_2021_tripdata_experiment') 

<Experiment: artifact_location='s3://mlflow-ride-duration21-prediction-artifact-store/1', creation_time=1753508117055, experiment_id='1', last_update_time=1753508117055, lifecycle_stage='active', name='nyc_greentaxi_2021_tripdata_experiment', tags={}>

In [32]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime)
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    df = df[((df['duration'] > 1) & (df['duration'] <= 60))]
    
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    return df

In [33]:
df_train = read_dataframe('green_tripdata/green_tripdata_2021-01.parquet')
df_val = read_dataframe('green_tripdata/green_tripdata_2021-02.parquet')

In [34]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

y_train = df_train['duration'].values
y_val = df_val['duration'].values

In [35]:
from math import sqrt

In [36]:
import xgboost as xgb

In [37]:
import mlflow.xgboost
from mlflow.models.signature import infer_signature
mlflow.xgboost.autolog(disable=True)

In [38]:
with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    val = xgb.DMatrix(X_val, label=y_val)

    mlflow.set_tag("engineer", "richkinwe")
    mlflow.log_param("train_data_path", "green_tripdata/green_tripdata_2021-01.parquet")
    mlflow.log_param("val_data_path", "green_tripdata/green_tripdata_2021-02.parquet")
    mlflow.set_tag("model", "xgboost")
    
    best_params = {
    'max_depth': 30,
    'learning_rate': 0.0959,
    'reg_alpha': 0.0181,
    'reg_lambda': 0.0117,
    'min_child_weight': 1.0606,
    'objective': 'reg:linear',
    'seed': 42
 }
    
    mlflow.log_params(best_params)

    booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=50,
            evals=[(val, 'validation')],
            early_stopping_rounds=50
        )
    
    y_pred = booster.predict(val)
    rmse = sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    with open('models.preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('models.preprocessor.b', artifact_path='preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path='models_xgboost_mlflow')

/Users/mac/Projects/MLops_pipeline/mlflow-311venv/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [06:38:12] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.44184
[1]	validation-rmse:10.76905
[2]	validation-rmse:10.18099
[3]	validation-rmse:9.67144
[4]	validation-rmse:9.22906
[5]	validation-rmse:8.84628
[6]	validation-rmse:8.51703
[7]	validation-rmse:8.23573
[8]	validation-rmse:7.99501
[9]	validation-rmse:7.78775
[10]	validation-rmse:7.61110
[11]	validation-rmse:7.45933
[12]	validation-rmse:7.33090
[13]	validation-rmse:7.22033
[14]	validation-rmse:7.12747
[15]	validation-rmse:7.04858
[16]	validation-rmse:6.98116
[17]	validation-rmse:6.92249
[18]	validation-rmse:6.87272
[19]	validation-rmse:6.82975
[20]	validation-rmse:6.79302
[21]	validation-rmse:6.76016
[22]	validation-rmse:6.73249
[23]	validation-rmse:6.70755
[24]	validation-rmse:6.68618
[25]	validation-rmse:6.66658
[26]	validation-rmse:6.64923
[27]	validation-rmse:6.63447
[28]	validation-rmse:6.62169
[29]	validation-rmse:6.60994
[30]	validation-rmse:6.60004
[31]	validation-rmse:6.59057
[32]	validation-rmse:6.58291
[33]	validation-rmse:6.57608
[34]	validation-rmse:

2025/07/26 06:38:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/mac/Projects/MLops_pipeline/mlflow-311venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [06:38:54] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/07/26 06:38:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run aged-frog-920 at: http://127.0.0.1:5003/#/experiments/1/runs/6625d3f1330a4418a6a73f9e17a4feeb
🧪 View experiment at: http://127.0.0.1:5003/#/experiments/1
